<a href="https://colab.research.google.com/github/a-emreusta/success-of-ai-writers/blob/main/Roberta_model_loaded.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
import gc
from google.colab import drive
drive.mount("/content/drive")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.4 MB/s 
     |████████████████████████████████| 182 kB 65.7 MB/s 
     |████████████████████████████████| 7.6 MB 50.9 MB/s 
Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import transformers

In [3]:
labels = ["contradiction", "entailment", "neutral"]
max_length = 128  # Maximum length of input sentence to the model.
batch_size = 32
epochs = 4
from tensorflow import keras

In [4]:
model = keras.models.load_model("/content/drive/MyDrive/Success of AI Writers/Colab/saved_model/my_model_roberta-base")

In [5]:
model.layers

 <keras.saving.saved_model.load.Custom>TFRobertaMainLayer at 0x7f97e973fd00>,

In [6]:
# train_df.head()
class RoBertaSemanticDataGenerator(tf.keras.utils.Sequence):
    """Generates batches of data.
    Args:
        sentence_pairs: Array of premise and hypothesis input sentences.
        labels: Array of labels.
        batch_size: Integer batch size.
        shuffle: boolean, whether to shuffle the data.
        include_targets: boolean, whether to incude the labels.
    Returns:
        Tuples `([input_ids, attention_mask, `token_type_ids], labels)`
        (or just `[input_ids, attention_mask, `token_type_ids]`
         if `include_targets=False`)
    """

    def __init__(
        self,
        sentence_pairs,
        labels,
        batch_size=batch_size,
        shuffle=True,
        include_targets=True,
    ):
        self.sentence_pairs = sentence_pairs
        self.labels = labels
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets
        # Load our BERT Tokenizer to encode the text.
        # We will use base-base-uncased pretrained model.
        self.tokenizer = transformers.RobertaTokenizer.from_pretrained(
            "roberta-large", do_lower_case=True
        )
        self.indexes = np.arange(len(self.sentence_pairs))
        self.on_epoch_end()

    def __len__(self):
        # Denotes the number of batches per epoch.
        return len(self.sentence_pairs) // self.batch_size

    def __getitem__(self, idx):
        # Retrieves the batch of index.
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        sentence_pairs = self.sentence_pairs[indexes]

        # With BERT tokenizer's batch_encode_plus batch of both the sentences are
        # encoded together and separated by [SEP] token.
        encoded = self.tokenizer.batch_encode_plus(
            sentence_pairs.tolist(),
            add_special_tokens=True,
            max_length=max_length,
            return_attention_mask=True,
            return_token_type_ids=True,
            pad_to_max_length=True,
            return_tensors="tf",
        )

        # Convert batch of encoded features to numpy array.
        input_ids = np.array(encoded["input_ids"], dtype="int32")
        attention_masks = np.array(encoded["attention_mask"], dtype="int32")
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

        # Set to true if data generator is used for training/validation.
        if self.include_targets:
            labels = np.array(self.labels[indexes], dtype="int32")
            return [input_ids, attention_masks, token_type_ids], labels
        else:
            return [input_ids, attention_masks, token_type_ids]

    def on_epoch_end(self):
        # Shuffle indexes after each epoch if shuffle is set to True.
        if self.shuffle:
            np.random.RandomState(42).shuffle(self.indexes)

In [7]:
def check_similarity(sentence1, sentence2):
    sentence_pairs = np.array([[str(sentence1), str(sentence2)]])
    test_data = RoBertaSemanticDataGenerator(
        sentence_pairs,
        labels=None,
        batch_size=1,
        shuffle=False,
        include_targets=False,
    )

    proba = model.predict(test_data[0])[0]
    #print(labels[0],"->",proba[0],"\n",labels[1],"->",proba[1],"\n",labels[2],"->",proba[2])
    idx = np.argmax(proba)
    proba = f"{proba[idx]: .2f}%"
    pred = labels[idx]
   
    return pred, proba

    # return proba

In [ ]:
sentence1 = "Two women are observing something together."
sentence2 = "Two women are standing with their eyes closed."
check_similarity(sentence1, sentence2)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 1s 538ms/step


('contradiction', ' 1.00%')

In [ ]:
sentence1 = "A soccer game with multiple males playing"
sentence2 = "Some men are playing a sport"
check_similarity(sentence1, sentence2)  

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 1s 536ms/step


('entailment', ' 0.90%')

In [ ]:
sentence1 = "A smiling costumed woman is holding an umbrella"
sentence2 = "A happy woman in a fairy costume holds an umbrella"
check_similarity(sentence1, sentence2)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 1s 549ms/step


('neutral', ' 0.97%')

In [ ]:
sentence1 = "102400000"
sentence2 = "104800000"
check_similarity(sentence1, sentence2)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 1s 606ms/step


('contradiction', ' 0.93%')

In [ ]:
with  open(r"/content/drive/MyDrive/Success of AI Writers/Copy_AI_Data/or_4.txt") as f:
  # or_data = [line.strip().split(".") for line in f.readlines()]
  or_data = f.readline()
f.close()

# print(or_data)

or_data_edited = or_data.split(". ")

with  open(r"/content/drive/MyDrive/Success of AI Writers/Copy_AI_Data/gpt_4.txt") as f:
  # ca_data = [line.split(".") for line in f.readlines()]
  ca_data = f.readline()
f.close()

ca_data_edited  = ca_data.split(". ")

print(len(ca_data_edited))
# print(ca_data_edited)
for line in ca_data_edited:
  print(line)


print(len(or_data_edited))
for line in or_data_edited:
  print(line)



17
Recent progress in large pre-trained language models has enabled the computational linguistics community to take on more knowledge-intensive tasks which require commonsense reasoning (Talmor et al., 2019; Bisk et al., 2020; Lin et al., 2021), and an understanding of figurative language (Pedinotti et al., 2021; Liu et al., 2022)
In this work, we are looking at a task related to figurative language understanding, known as euphemism detection
Euphemisms are used to soften or replace harsh, impolite, or blunt expressions about taboo or sensitive topics such as death and unemployment (Holder, 2008)
For example, when referring to older people, we often use the term 'senior citizens' instead of a more direct term that could be seen as offensive
Identifying euphemisms is difficult due to their natural ambiguity, i.e
the meaning of the term changes depending on the context: 'Over the hill' could either mean someone or something is physically over a hill (literal), or someone or something is 

In [ ]:
sim_matrix=np.zeros((len(or_data_edited),len(ca_data_edited)))

In [ ]:
for or_idx, o_line in enumerate(or_data_edited):
  print(o_line)
  for ca_idx, c_line in enumerate(ca_data_edited):
    if c_line != "" and o_line != "":
      (plg_type,sm_ratio)=(check_similarity(o_line.strip(), c_line.strip()))
      print(plg_type,sm_ratio)
      if plg_type == "entailment":
        sim_matrix[or_idx][ca_idx]=float(sm_ratio[:-1])
    else:
      print("bong")
  print()
  # 6 dk

Moussaka is one of the best known Greek dishes – a baked casserole consisting of ground lamb meat and layers of sliced eggplant, covered with a thick layer of bechamel sauce that gets golden and crusty as it bakes


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


1/1 [==============================] - 1s 910ms/step
contradiction -> 0.0019718495 
 entailment -> 0.58673984 
 neutral -> 0.41128832
entailment  0.59%


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 1s 846ms/step
contradiction -> 0.004674227 
 entailment -> 0.109702006 
 neutral -> 0.88562375
neutral  0.89%


...

1/1 [==============================] - 1s 511ms/step
contradiction -> 0.004276522 
 entailment -> 0.49433452 
 neutral -> 0.501389
neutral  0.50%


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 1s 531ms/step
contradiction -> 0.0559384 
 entailment -> 0.00603397 
 neutral -> 0.9380277
neutral  0.94%


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 1s 534ms/step
contradiction -> 0.013045406 
 entailment -> 0.9473825 
 neutral -> 0.03957206
entailment  0.95%



In [ ]:
df_sm_matrix=pd.DataFrame(sim_matrix)
df_sm_matrix

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0.71,0.00,0.0,0.00,0.00,0.55,0.00,0.00,0.00,0.00,0.48,0.00,0.00,0.00,0.00,0.00,0.0
1,0.00,0.88,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
2,0.00,0.95,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.47,0.00,0.00,0.0
3,0.00,0.00,0.0,0.73,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
4,0.00,0.83,0.0,0.00,0.70,0.00,0.53,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
5,0.00,0.00,0.0,0.00,0.00,0.00,0.97,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
6,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.91,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
7,0.00,0.96,0.0,0.00,0.78,0.49,0.00,0.63,0.59,0.00,0.00,0.00,0.66,0.00,0.00,0.00,0.0
8,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.83,0.00,0.00,0.00,0.00,0.00,0.00,0.0
9,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.94,0.00,0.00,0.00,0.00,0.00,0.0


In [ ]:
max(df_sm_matrix[0])

0.71

In [ ]:
sim_matrix[1]

array([0.  , 0.88, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  ])

In [95]:
with  open(r"/content/drive/MyDrive/Success of AI Writers/Copy_AI_Data/musakka.txt") as f:
  # or_data = [line.strip().split(".") for line in f.readlines()]
  or_data = f.readline()
f.close()

# print(or_data)

or_data_edited = or_data.split(". ")

with  open(r"/content/drive/MyDrive/Success of AI Writers/Copy_AI_Data/gpt_musakka.txt") as f:
  # ca_data = [line.split(".") for line in f.readlines()]
  ca_data = f.readline()
f.close()

ca_data_edited  = ca_data.split(". ")

print(len(ca_data_edited))
# print(ca_data_edited)
for i,line in enumerate(ca_data_edited):
  print("ca"+str(i+1)+"-->",line)


print(len(or_data_edited))
for i,line in enumerate(or_data_edited):
  print("or"+str(i+1)+"-->",line)



5
ca1--> Moussaka is a traditional Greek dish made up of ground lamb (or sometimes beef) and layers of eggplant, all covered in a thick and golden bechamel sauce
ca2--> It is thought to have originated from the Middle East, as the name is derived from the Turkish musakka, which comes from the Arabic word musaqqa'ah, meaning chilled
ca3--> It is not a meal eaten every day, but is a special treat that is served on festive occasions
ca4--> It is similar to lasagna, but without the pasta, and is very healthy due to the vegetables used
ca5--> It is usually cut into squares and served warm after it has had some time to cool and firm up.
6
or1--> Moussaka is one of the best known Greek dishes – a baked casserole consisting of ground lamb meat and layers of sliced eggplant, covered with a thick layer of bechamel sauce that gets golden and crusty as it bakes
or2--> The lamb is sometimes replaced with beef, while the eggplants might be replaced with zucchini or potatoes
or3--> It is likely that 

In [96]:
def create_sim_matrix(original,copy_ai):
  sim_matrix=np.zeros((len(original),len(copy_ai)))
  for or_idx, o_line in enumerate(original):
     print(o_line)
     for ca_idx, c_line in enumerate(copy_ai):
        if c_line != "" and o_line != "":
          (plg_type,sm_ratio)=(check_similarity(o_line.strip(), c_line.strip()))
          print(plg_type,sm_ratio)
          if plg_type == "entailment":
              sim_matrix[or_idx][ca_idx]=float(sm_ratio[:-1])
        else:
          print("bong")
     print()   
  return pd.DataFrame(sim_matrix,columns=["ca"+str(i) for i in range(1,len(copy_ai)+1)],index=["or"+str(i) for i in range(1,len(original)+1)])

In [97]:
df_sm_matrix=create_sim_matrix(or_data_edited,ca_data_edited)

Moussaka is one of the best known Greek dishes – a baked casserole consisting of ground lamb meat and layers of sliced eggplant, covered with a thick layer of bechamel sauce that gets golden and crusty as it bakes


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


1/1 [==============================] - 1s 638ms/step
entailment  0.59%


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 1s 601ms/step
neutral  0.89%


...

1/1 [==============================] - 1s 583ms/step
neutral  0.50%


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 1s 578ms/step
neutral  0.94%


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 1s 581ms/step
entailment  0.95%



In [89]:
df_sm_matrix

,ca1,ca2,ca3,ca4,ca5
or1,0.59,0.0,0.71,0.00,0.00
or2,0.00,0.0,0.00,0.00,0.00
or3,0.00,0.0,0.54,0.00,0.00
or4,0.00,0.0,0.88,0.00,0.00
or5,0.00,0.0,0.64,0.94,0.00
or6,0.00,0.0,0.00,0.00,0.95


In [92]:
def save_result(sm_matrix,or_,ca_,file_name):
  try:
      f = open("/content/drive/MyDrive/Success of AI Writers/Copy_AI_Similarity/"+file_name+".txt", "w")
      for i in range(df_sm_matrix.shape[0]):
        for j in range(df_sm_matrix.shape[1]):
          if df_sm_matrix.iloc[i,j]!=0:
            f.write(df_sm_matrix.index[i]+":"+or_data_edited[i]+"\n"+df_sm_matrix.columns[j]+":"+ca_data_edited[j]+"\n"+df_sm_matrix.index[i]+" - "+df_sm_matrix.columns[j]+" Similarity ratio : "+ str(df_sm_matrix.iloc[i,j])+"\n")
  except:
     print("Something went wrong when opening the file")
  finally:
    f.close()


In [93]:
save_result(df_sm_matrix,or_data_edited,ca_data_edited,"musakka2")

In [ ]:
with  open(r"/content/drive/MyDrive/Colab Notebooks/or2.txt") as f:
  # or_data = [line.strip().split(".") for line in f.readlines()]
  or_data = f.readline()
f.close()

# print(or_data)

or_data_edited = or_data.split(".")
print(len(or_data_edited))

with  open(r"/content/drive/MyDrive/Colab Notebooks/ca2.txt") as f:
  # ca_data = [line.split(".") for line in f.readlines()]
  ca_data = f.readline()
f.close()

ca_data_edited  = ca_data.split(".")
print(len(ca_data_edited))

12
11


In [ ]:
for or_idx, o_line in enumerate(or_data_edited):
  print(o_line)
  for ca_idx, c_line in enumerate(ca_data_edited):
    if c_line != "" and o_line != "":
      print(check_similarity(o_line.strip(), c_line.strip()))
    
    else:
      print("bong")
  
  print()

Language models have shown both quantitative improvement and new qualitative capabilities as they increase in size


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


1/1 [==============================] - 1s 907ms/step
('entailment', ' 0.91%')


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 1s 900ms/step
('entailment', ' 0.71%')


...

1/1 [==============================] - 1s 597ms/step
('contradiction', ' 0.46%')


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 1s 591ms/step
('neutral', ' 0.45%')


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 1s 580ms/step
('neutral', ' 0.65%')
bong


bong
bong
bong
bong
bong
bong
bong
bong
bong
bong
bong



In [ ]:
with  open(r"/content/drive/MyDrive/Colab Notebooks/or1.txt") as f:
  # or_data = [line.strip().split(".") for line in f.readlines()]
  or_data = f.readline()
f.close()

# print(or_data)

or_data_edited = or_data.split(".")
print(len(or_data_edited))

with  open(r"/content/drive/MyDrive/Colab Notebooks/or2.txt") as f:
  # ca_data = [line.split(".") for line in f.readlines()]
  ca_data = f.readline()
f.close()

ca_data_edited  = ca_data.split(".")
print(len(ca_data_edited))

8
12


In [ ]:
for or_idx, o_line in enumerate(or_data_edited):
  print(o_line)
  for ca_idx, c_line in enumerate(ca_data_edited):
    if c_line != "" and o_line != "":
      print(check_similarity(o_line.strip(), c_line.strip()))
    
    else:
      print("bong")
  
  print()

This paper describes our two-stage system1 for the Euphemism Detection shared task hosted by the 3rd Workshop on Figurative Language Processing in conjunction with EMNLP 2022


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 1s 975ms/step
('entailment', ' 0.71%')


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 1s 948ms/step
('contradiction', ' 0.70%')


...

1/1 [==============================] - 1s 600ms/step
('entailment', ' 0.48%')


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 1s 578ms/step
('contradiction', ' 0.53%')


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 1s 562ms/step
('entailment', ' 0.46%')
bong



In [ ]:
with  open(r"/content/drive/MyDrive/Colab Notebooks/or2.txt") as f:
  # or_data = [line.strip().split(".") for line in f.readlines()]
  or_data = f.readline()
f.close()

# print(or_data)

or_data_edited = or_data.split(".")
print(len(or_data_edited))

with  open(r"/content/drive/MyDrive/Colab Notebooks/musakka.txt") as f:
  # ca_data = [line.split(".") for line in f.readlines()]
  ca_data = f.readline()
f.close()

ca_data_edited  = ca_data.split(".")
print(len(ca_data_edited))

12
8


In [ ]:
for or_idx, o_line in enumerate(or_data_edited):
  print(o_line)
  for ca_idx, c_line in enumerate(ca_data_edited):
    if c_line != "" and o_line != "":
      print(check_similarity(o_line.strip(), c_line.strip()))
    
    else:
      print("bong")
  
  print()

Language models have shown both quantitative improvement and new qualitative capabilities as they increase in size


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 1s 583ms/step
('neutral', ' 0.63%')


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 1s 596ms/step
('contradiction', ' 1.00%')


...

1/1 [==============================] - 1s 575ms/step
('contradiction', ' 0.70%')


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 1s 565ms/step
('neutral', ' 0.77%')


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 1s 572ms/step
('contradiction', ' 0.78%')
bong


bong
bong
bong
bong
bong
bong
bong
bong

